# Regresiones lineales



In [2]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [3]:
# Configuraciones
warnings.filterwarnings('ignore')
# Reduzcamos el número de línea a leer
pd.options.display.max_rows = 5

In [10]:
# Años de análisis
period = 11
csv = '.csv'
# Directorios
pitcher_path = '../../Data/New_Data/Free_Agent/Pitchers/complete_free_agents_pitchers_'
hitter_path = '../../Data/New_Data/Free_Agent/Hitters/complete_free_agents_hitters_'
# Originales:
df_pitcher = [None]*period
df_hitter = [None]*period

In [11]:
for i in range(0,period):
    df_pitcher[i] = pd.read_csv(pitcher_path + str(2011 + i) + csv)
    df_hitter[i] = pd.read_csv(hitter_path + str(2011 + i) + csv)    

Veamos el tipo de datos contenidos en las bases de datos

In [15]:
df_pitcher[2].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Juegos_iniciados,Inning_pitched,Bateos,Carreras,Carreras_ganadas,...,Wins,Losses,Saves,WHIP,ERA,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias
0,Zack Greinke,147000000,24500000,SP,29,28,177.7,152,54,52,...,15,4,0,1.11,2.63,LAD,19000000,92,162,0.567901
1,J.P. Howell,2850000,2850000,RP,67,0,62.0,42,14,14,...,4,1,0,1.05,2.03,LAD,2850000,92,162,0.567901
2,Brian Wilson,1000000,1000000,RP,18,0,13.7,8,1,1,...,2,1,0,0.88,0.66,LAD,1000000,92,162,0.567901
3,Peter Moylan,0,0,RP,14,0,15.3,23,11,11,...,1,0,0,1.96,6.46,LAD,490000,92,162,0.567901
4,Anibal Sanchez,80000000,16000000,RP,29,29,182.0,156,56,52,...,14,8,0,1.15,2.57,DET,8800000,93,162,0.574074


In [17]:
df_hitter[7].head()

,Jugador,Valor_contrato,Valor_promedio_contrato,Posicion,Juegos,Porcetnaje_juegos,At-bats,Bateos,Home-runs,RBI,Porcentaje_bateo,OPS,Equipo,Sueldo,Victorias,Juegos totales,Promedio_victorias
0,Eric Hosmer,144000000,18000000,1B,157,0.969,613,155,18,69,0.253,0.720,SD,21000000,66,162,0.407407
1,Yu Darvish,126000000,21000000,SP,8,0.049,12,1,0,0,0.083,0.250,CHC,25000000,95,163,0.582822
2,Tyler Chatwood,38000000,12666667,SP,25,0.153,25,4,0,2,0.160,0.352,CHC,12500000,95,163,0.582822
3,Brandon Morrow,21000000,10500000,RP,35,0.215,1,0,0,0,0.000,0.000,CHC,9000000,95,163,0.582822
4,Steve Cishek,13000000,6500000,RP,81,0.497,6,1,0,0,0.167,0.333,CHC,7500000,95,163,0.582822


Debido a la enorme diferencias en la magnitud entre el promedio de victorias y los sueldos, se tendrá que estadandarizar los sueldos.

In [18]:
# Originales:
df_pitcher_copy = [None]*period
df_hitter_copy = [None]*period

In [19]:
for i in range(0,period):
    df_pitcher_copy[i] = df_pitcher[i].copy()
    df_hitter_copy[i] = df_hitter[i].copy()
    
    df_pitcher_copy[i]['Sueldo_estandarizado'] = df_pitcher_copy[i]['Sueldo']/df_pitcher_copy[i]['Sueldo'].max()
    df_hitter_copy[i]['Sueldo_estandarizado'] = df_hitter_copy[i]['Sueldo']/df_hitter_copy[i]['Sueldo'].max()

Ahora, tanto la variable dependiente como independiente están en el intervalo $[0,1]$

In [21]:
df_pitcher_copy[2][['Sueldo_estandarizado', 'Promedio_victorias']].head()

,Sueldo_estandarizado,Promedio_victorias
0,1.000000,0.567901
1,0.150000,0.567901
2,0.052632,0.567901
3,0.025789,0.567901
4,0.463158,0.574074


In [22]:
df_hitter_copy[2][['Sueldo_estandarizado', 'Promedio_victorias']].head()

,Sueldo_estandarizado,Promedio_victorias
0,1.000000,0.567901
1,0.150000,0.567901
2,0.915789,0.481481
3,0.342105,0.481481
4,0.463158,0.574074


## I. Victorias en función del salario

Para cada año de an{alisis, consideraremos el promedio de victorias como medida del esfuerzo y el total del salario

Creemos una lista para guardar los modelos y proceder a obtenerlos

In [35]:
# Modelos
hitter_model_I = [None]*period
pitcher_model_I = [None]*period

In [43]:
for i in range(1,period):
    # Hitters:
    y_hitter = df_hitter_copy[i]['Promedio_victorias'].tolist()
    x_hitter = df_hitter_copy[i]['Sueldo_estandarizado'].tolist()
    
    x_hitter = sm.add_constant(x_hitter)
    
    hitter_model_I[i] = sm.OLS(y_hitter, x_hitter).fit()
    
    # Pitchers:
    y_pitcher = df_pitcher_copy[i]['Promedio_victorias'].tolist()
    x_pitcher = df_pitcher_copy[i]['Sueldo_estandarizado'].tolist()
    
    x_pitcher = sm.add_constant(x_pitcher)
    
    pitcher_model_I[i] = sm.OLS(y_pitcher, x_pitcher).fit()

Observemos los resúmenes de las regresiones

#### Hitters

In [40]:
for i in range(1,period):
    print(hitter_model_I[i].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.021
Method:                 Least Squares   F-statistic:                  0.001698
Date:                Wed, 29 Jun 2022   Prob (F-statistic):              0.967
Time:                        01:04:31   Log-Likelihood:                 65.615
No. Observations:                  49   AIC:                            -127.2
Df Residuals:                      47   BIC:                            -123.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5192      0.013     39.121      0.0

Los resultados muestran que, en general, hay poca significancia estadística para explicar las victorias con el salario de los agentes libres. Así mismo, tampoco tiene poder predictivo puesto que la $R^2$ es baja. Faltará corroborar esto con validación cruzada, sin embargo, el hecho de la baja significancia estadística no cambiará.

#### Pitchers

In [44]:
for i in range(1,period):
    print(pitcher_model_I[i].summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.051
Method:                 Least Squares   F-statistic:                    0.1269
Date:                Wed, 29 Jun 2022   Prob (F-statistic):              0.726
Time:                        01:05:24   Log-Likelihood:                 25.191
No. Observations:                  19   AIC:                            -46.38
Df Residuals:                      17   BIC:                            -44.49
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5194      0.027     19.110      0.0

Observamos los mismos problemas.

De esto, podemos concluir que este modelo no es el más adecuado, por lo que ahora se intentará con el resto de medidas de esfuerzo respectivamente para *hitters* y *pitchers*.

## Salario en función de las medidas de esfuerzo

Lo que se hará es regresar sobre todas las medidas del esfuerzo con el objetivo de determinar cuáles variables son significativas para el modelo y cuáles no